# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo.uk_food

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db.establishments

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": {"$eq": 20}}

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6415e588677f94754952b0e7'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Decimal128('0.27694')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta

In [7]:
# Convert the result to a Pandas DataFrame
results_df = pd.json_normalize(establishments.find(query))
# Display the number of rows in the DataFrame
print(len(results_df))

# Display the first 10 rows of the DataFrame
results_df.head(10)

41


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,6415e588677f94754952b0e7,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,...,0.27694,50.769705,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,6415e588677f94754952b46a,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,1.278721,51.857536,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,6415e588677f94754952b771,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,1.15927,51.789429,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,6415e588677f94754952b962,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,0.10202,50.770885,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,6415e588677f94754952b971,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,0.101446,50.770724,None,0001-01-01T00:00:00,0,None,0,0,0,0
5,6415e588677f94754952c30f,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,...,0.736349,51.541448,None,0001-01-01T00:00:00,0,None,0,0,0,0
6,6415e588677f94754952c534,1537089,0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,...,0.7121671,51.5350065,None,0001-01-01T00:00:00,0,None,0,0,0,0
7,6415e589677f94754952da5c,155648,0,EH/00006058,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,...,0.602364,51.591515,None,0001-01-01T00:00:00,0,None,0,0,0,0
8,6415e589677f94754952dea2,1012883,0,17/00110/MIXED/S,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,...,0.146449998021126,51.1859474182129,None,0001-01-01T00:00:00,0,None,0,0,0,0
9,6415e589677f94754952e6b1,644109,0,41811,F & S,Retailers - other,4613,,81 Southernhay,Basildon,...,0.462307,51.57005,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Convert RatingValue to integer wherever possible.
establishments.update_many({}, [ {"$set": {"RatingValue": {"$convert": {"input": "$RatingValue", "to": "int", "onError": "$RatingValue"}}}} ])

# Change the data type from String to Decimal for longitude
establishments.update_many({}, [ {"$set": {"geocode.longitude": {"$toDecimal": "$geocode.longitude"}}} ])

# Change the data type from String to Decimal for latitude
establishments.update_many({}, [ {"$set": {"geocode.latitude": {"$toDecimal": "$geocode.latitude"}}} ])

In [9]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London"}, "RatingValue":  {"$gte": 4}}

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

33
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6415e588677f94754952cb02'),
 'geocode': {'latitude': Decimal128('51.369321'),
             'longitude': Decimal128('0.508551')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establi

In [10]:
# Convert the result to a Pandas DataFrame
london_results_df = pd.json_normalize(establishments.find(query))

# Display the number of rows in the DataFrame
print(len(london_results_df))

# Display the first 10 rows of the DataFrame
london_results_df.head(10)

33


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,6415e588677f94754952cb02,621707,0,PI/000025307,Charlie's,Other catering premises,7841,Oak Apple Farm Building 103 Sheernes Docks,Sheppy Kent,,...,0.508551,51.369321,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,6415e588677f94754952ce28,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,0.249255999922752,51.1236228942871,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,6415e589677f94754952d984,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,0.561954,51.543831,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,6415e589677f94754952e775,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,0.346336990594864,51.4640655517578,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,6415e589677f94754952e777,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,0.346325,51.464078,None,0001-01-01T00:00:00,0,None,0,0,0,0
5,6415e589677f94754952ee7e,294900,0,PI/000019066,Mv Valulla,Other catering premises,7841,Reeds River Cruises LtdKings ReachRiver Thames...,London,,...,0.287555,51.504071,None,0001-01-01T00:00:00,0,None,0,0,0,0
6,6415e589677f947549530f90,293756,0,PI/000002538,Tereza Joanne,Other catering premises,7841,Funcraft UK Ltd King George V Dock Woolwich Ma...,London,,...,0.069286,51.501121,None,0001-01-01T00:00:00,0,None,0,0,0,0
7,6415e589677f947549531353,878523,0,PI/000029844,Brick Lane Brews,Restaurant/Cafe/Canteen,1,Air side London City Airport,London,,...,0.048006,51.503733,None,0001-01-01T00:00:00,0,None,0,0,0,0
8,6415e589677f94754953136d,293772,0,PI/000002585,The Nuance Group (UK) Limited,Retailers - other,4613,Duty Free Shop Passenger Terminal,London City AirportRoyal DocksLondon,,...,0.047966,51.504112,None,0001-01-01T00:00:00,0,None,0,0,0,0
9,6415e589677f94754953136e,294606,0,PI/000016185,WH Smith,Retailers - other,4613,London City Airport,Silvertown London,,...,0.0479655750095844,51.5041122436523,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [15]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
# Latitude and Longitude of Greenwich, where the newly added restaurant "Penang Flavours" is located.
latitude = 51.4934
longitude = 0.0098

query = {
    "RatingValue": {
        "$eq": 5
    },
    "geocode.latitude": {
        "$lte": latitude + degree_search,
        "$gte": latitude - degree_search
    },
    "geocode.longitude": {
        "$lte": longitude + degree_search,
        "$gte": longitude - degree_search
    }
}
sort = [("scores.Hygiene", 1)]

print(establishments.count_documents(query))
# Print the results
results = establishments.find(query).sort(sort).limit(5)
for result in results:
    pprint(result)

146
{'AddressLine1': 'The Angerstein 108 Woolwich Road',
 'AddressLine2': '',
 'AddressLine3': 'Greenwich',
 'AddressLine4': '',
 'BusinessName': 'Angerstein Hotel',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4649.649445942743,
 'FHRSID': 695486,
 'LocalAuthorityBusinessID': 'PI/000186740',
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE10 0LE',
 'RatingDate': '2021-06-19T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6415e589677f9475495315ec'),
 'geocode': {'latitude': Decimal128('51.4863131'),
             'longitude': Decimal128('0.0174768')},
 'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/695486',
            'rel': 'self'}],
 'meta'

In [12]:
# Convert result to Pandas DataFrame
greenwich_results_df = pd.json_normalize(results)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [13]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
query = {"$match": {"scores.Hygiene": {"$eq": 0}}}
# 2. Groups the matches by Local Authority
group = {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}}
# 3. Sorts the matches from highest to lowest
sort = {"$sort": {"count": -1}}

pipeline = [query, group, sort]
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(len(results))

# Print the first 10 results
pprint(results[0:10])

55
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [14]:
# Convert the result to a Pandas DataFrame
zero_hygiene_df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(zero_hygiene_df.count())

# Display the first 10 rows of the DataFrame
zero_hygiene_df.head(10)

_id      55
count    55
dtype: int64


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
